# How to backtest 2,000,000 simulations for the best exits

A backtest is a simulation of market dynamics that are used to test how trading strategies might perform.

The problem is that the markets are noisy and difficult to model.

So when running backtests, optimized models often fit to noise, instead of market inefficiencies. When applying the optimized parameters to previously unseen data, the model falls apart (along with your portfolio).

All parameters of a strategy affect the result but only a few determine entry and exit dependent on the market price. These are the parameters that should be optimized.

For example, is a 1% trailing stop better than a $1 stop loss?

We’ll use the cutting-edge backtesting framework vectorbt to optimize the entry and exit type for a momentum strategy.

In [2]:
import pytz
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import vectorbt as vbt

In [3]:
symbols = [
    "META",
    "AMZN",
    "AAPL",
    "NFLX",
    "GOOG",
]

start_date = datetime(2018, 1, 1, tzinfo=pytz.utc)
end_date = datetime(2021, 1, 1, tzinfo=pytz.utc)

traded_count = 3
window_len = timedelta(days=12 * 21)

seed = 42
window_count = 400
exit_types = ["SL", "TS", "TP"]
stops = np.arange(0.01, 1 + 0.01, 0.01)

In [4]:
yfdata = vbt.YFData.download(symbols, start=start_date, end=end_date)
ohlcv = yfdata.concat()

split_ohlcv = {}

for k, v in ohlcv.items():
    split_df, split_indexes = v.vbt.range_split(
        range_len=window_len.days, n=window_count
    )
    split_ohlcv[k] = split_df
ohlcv = split_ohlcv

## Build the momentum strategy

Our strategy selects the top 3 stocks every split based on their mean return. The strategy equally allocates across the stocks at the beginning of the period, and exits at the end.

In [5]:
momentum = ohlcv["Close"].pct_change().mean()

sorted_momentum = (
    momentum
    .groupby(
        "split_idx", 
        group_keys=False, 
        sort=False
    )
    .apply(
        pd.Series.sort_values
    )
    .groupby("split_idx")
    .head(traded_count)
)

selected_open = ohlcv["Open"][sorted_momentum.index]
selected_high = ohlcv["High"][sorted_momentum.index]
selected_low = ohlcv["Low"][sorted_momentum.index]
selected_close = ohlcv["Close"][sorted_momentum.index]